In [1]:
# import warnings
# warnings.filterwarnings('ignore')
%load_ext autoreload
%autoreload 2

import pickle
import numpy as np
import scipy.sparse as sp
from scipy.sparse import load_npz
import pandas as pd

import torch

from cell import utils
from cell.utils import link_prediction_performance, edge_overlap
from cell.cell import Cell, EdgeOverlapCriterion, LinkPredictionCriterion
from cell.graph_statistics import compute_graph_statistics

In [31]:
def run(g_type='cell', loss_fn=None, criterion='eo', nsteps=200, nsamples=5, optimizer_args=None, collect_stat=True):
    if criterion == 'eo':
        callbacks = [EdgeOverlapCriterion(invoke_every=10, edge_overlap_limit=.5)]
    elif criterion == 'val':
        callbacks = [LinkPredictionCriterion(invoke_every=2,
                                                val_ones=val_ones,
                                            val_zeros=val_zeros,
                                                max_patience=5)]
    else:
        raise NameError

    model = Cell(A=train_graph,
             H=9,
             g_type=g_type,
             loss_fn=loss_fn,
             callbacks=callbacks)

    # train model
    if optimizer_args is None:
         optimizer_args = {'lr': 0.1, 'weight_decay': 1e-5}
    model.train(steps=nsteps,
                optimizer_fn=torch.optim.Adam,
                optimizer_args=optimizer_args)
    
    roc_auc, avg_prec = link_prediction_performance(scores_matrix=model._scores_matrix, val_ones=test_ones, val_zeros=test_zeros)
    print(f'ROC-AUC: {roc_auc}, AVG_PREC: {avg_prec}')

    if collect_stat:
        generated_graphs = [model.sample_graph() for _ in range(nsamples)]
        stats = [compute_graph_statistics(gg) for gg in generated_graphs]
        stat_df = pd.DataFrame({k: [s[k] for s in stats] for k in stats[0].keys()})
        stat_df = stat_df.mean()
        stat_df['roc-auc'] = roc_auc
        stat_df['avg-prec'] = avg_prec

        return stat_df.T, generated_graphs

# CORA ML

In [5]:
#train_graph 
_A_obs, _X_obs, _z_obs = utils.load_npz('../data/cora_ml.npz')
_A_obs = _A_obs + _A_obs.T
_A_obs[_A_obs > 1] = 1
lcc = utils.largest_connected_components(_A_obs)
_A_obs = _A_obs[lcc,:][:,lcc]
_N = _A_obs.shape[0]

Selecting 1 largest connected components


In [6]:
val_share = 0.05
test_share = 0.1
seed = 42 #481516234

train_ones, val_ones, val_zeros, test_ones, test_zeros = utils.train_val_test_split_adjacency(_A_obs, val_share, test_share, seed, undirected=True, connected=True, asserts=True)

train_graph = sp.coo_matrix((np.ones(len(train_ones)),(train_ones[:,0], train_ones[:,1]))).tocsr()
assert (train_graph.toarray() == train_graph.toarray().T).all()

In [5]:
df = pd.DataFrame()

### Edge overlap

#### CELL

In [6]:
method = 'cell'
options = dict(g_type='cell')

stat_df, _ = run(**options)
df[method] = stat_df

Step:  10/200 Loss: 6.03302 Edge-Overlap: 0.033 Total-Time: 4
Step:  20/200 Loss: 4.08839 Edge-Overlap: 0.278 Total-Time: 9
Step:  30/200 Loss: 3.32947 Edge-Overlap: 0.415 Total-Time: 14
Step:  40/200 Loss: 3.00776 Edge-Overlap: 0.499 Total-Time: 18
Step:  50/200 Loss: 2.85692 Edge-Overlap: 0.532 Total-Time: 23
ROC-AUC: 0.9412563576811439, AVG_PREC: 0.9458396278486313


#### SVD

In [7]:
method = 'svd'
options = dict(g_type='svd', nsteps=400)

stat_df, _ = run(**options)
df[method] = stat_df

Step:  10/400 Loss: 7.93868 Edge-Overlap: 0.002 Total-Time: 4
Step:  20/400 Loss: 7.50294 Edge-Overlap: 0.018 Total-Time: 8
Step:  30/400 Loss: 6.52674 Edge-Overlap: 0.047 Total-Time: 13
Step:  40/400 Loss: 5.74590 Edge-Overlap: 0.095 Total-Time: 17
Step:  50/400 Loss: 5.30830 Edge-Overlap: 0.121 Total-Time: 22
Step:  60/400 Loss: 5.04991 Edge-Overlap: 0.147 Total-Time: 26
Step:  70/400 Loss: 4.89850 Edge-Overlap: 0.153 Total-Time: 31
Step:  80/400 Loss: 4.79412 Edge-Overlap: 0.164 Total-Time: 35
Step:  90/400 Loss: 4.72522 Edge-Overlap: 0.168 Total-Time: 40
Step: 100/400 Loss: 4.67763 Edge-Overlap: 0.179 Total-Time: 44
Step: 110/400 Loss: 4.64200 Edge-Overlap: 0.168 Total-Time: 48
Step: 120/400 Loss: 4.64415 Edge-Overlap: 0.178 Total-Time: 53
Step: 130/400 Loss: 4.65872 Edge-Overlap: 0.182 Total-Time: 57
Step: 140/400 Loss: 4.64247 Edge-Overlap: 0.188 Total-Time: 62
Step: 150/400 Loss: 4.61682 Edge-Overlap: 0.187 Total-Time: 66
Step: 160/400 Loss: 4.64835 Edge-Overlap: 0.186 Total-Tim

#### our CELL

In [8]:
method = 'nonlin cell'
optimizer_args = dict(lr=0.1, weight_decay=1e-4)
options = dict(g_type='fc', optimizer_args=optimizer_args)

stat_df, _ = run(**options)
df[method] = stat_df

Step:  10/200 Loss: 7.36982 Edge-Overlap: 0.021 Total-Time: 5
Step:  20/200 Loss: 6.98611 Edge-Overlap: 0.027 Total-Time: 10
Step:  30/200 Loss: 6.31122 Edge-Overlap: 0.076 Total-Time: 15
Step:  40/200 Loss: 5.33766 Edge-Overlap: 0.114 Total-Time: 20
Step:  50/200 Loss: 4.77716 Edge-Overlap: 0.158 Total-Time: 25
Step:  60/200 Loss: 4.02139 Edge-Overlap: 0.278 Total-Time: 30
Step:  70/200 Loss: 3.69880 Edge-Overlap: 0.335 Total-Time: 35
Step:  80/200 Loss: 3.37106 Edge-Overlap: 0.415 Total-Time: 40
Step:  90/200 Loss: 3.31426 Edge-Overlap: 0.427 Total-Time: 45
Step: 100/200 Loss: 3.12393 Edge-Overlap: 0.465 Total-Time: 49
Step: 110/200 Loss: 2.92657 Edge-Overlap: 0.543 Total-Time: 54
ROC-AUC: 0.9615993709283037, AVG_PREC: 0.9630098117109628


#### Results

In [9]:
df['gt'] = list(compute_graph_statistics(_A_obs).values()) + [1, 1]

In [10]:
df

,cell,svd,nonlin cell,gt
d_max,1.982000e+02,1.818000e+02,2.146000e+02,2.460000e+02
d_min,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
d,4.827758e+00,4.827758e+00,4.827758e+00,5.680427e+00
LCC,2.805400e+03,2.810000e+03,2.801400e+03,2.810000e+03
wedge_count,8.186840e+04,7.875720e+04,8.601620e+04,1.377190e+05
claw_count,1.805467e+06,1.479182e+06,2.181900e+06,3.930163e+06
triangle_count,1.352200e+03,1.120800e+03,1.626000e+03,5.247000e+03
square_count,7.024800e+03,4.708400e+03,9.221400e+03,3.450700e+04
power_law_exp,1.816033e+00,1.827045e+00,1.823022e+00,1.767268e+00
gini,4.473638e-01,4.551560e-01,4.531931e-01,4.964733e-01


### Link Prediction Criterion

#### CELL

In [11]:
method = 'cell'
options = dict(g_type='cell', criterion='val', collect_stat=False)

run(**options)
#df[method] = stat_df

Step:   2/200 Loss: 7.92950 ROC-AUC Score: 0.584 Average Precision: 0.582 Total-Time: 0
Step:   4/200 Loss: 7.73384 ROC-AUC Score: 0.650 Average Precision: 0.663 Total-Time: 1
Step:   6/200 Loss: 7.32220 ROC-AUC Score: 0.697 Average Precision: 0.720 Total-Time: 1
Step:   8/200 Loss: 6.71251 ROC-AUC Score: 0.746 Average Precision: 0.773 Total-Time: 2
Step:  10/200 Loss: 6.02657 ROC-AUC Score: 0.797 Average Precision: 0.826 Total-Time: 3
Step:  12/200 Loss: 5.41900 ROC-AUC Score: 0.844 Average Precision: 0.870 Total-Time: 4
Step:  14/200 Loss: 4.96410 ROC-AUC Score: 0.879 Average Precision: 0.899 Total-Time: 4
Step:  16/200 Loss: 4.60738 ROC-AUC Score: 0.897 Average Precision: 0.913 Total-Time: 5
Step:  18/200 Loss: 4.30933 ROC-AUC Score: 0.905 Average Precision: 0.919 Total-Time: 6
Step:  20/200 Loss: 4.06496 ROC-AUC Score: 0.910 Average Precision: 0.924 Total-Time: 7
Step:  22/200 Loss: 3.86109 ROC-AUC Score: 0.917 Average Precision: 0.929 Total-Time: 8
Step:  24/200 Loss: 3.69987 ROC-

#### SVD

In [7]:
method = 'svd'
options = dict(g_type='svd', criterion='val', nsteps=400, collect_stat=False)

run(**options)
#df[method] = stat_dfmethod = 'cell'

Step:   2/400 Loss: 7.94522 ROC-AUC Score: 0.489 Average Precision: 0.490 Total-Time: 0
Step:   4/400 Loss: 7.94192 ROC-AUC Score: 0.498 Average Precision: 0.495 Total-Time: 1
Step:   6/400 Loss: 7.94159 ROC-AUC Score: 0.483 Average Precision: 0.537 Total-Time: 2
Step:   8/400 Loss: 7.94049 ROC-AUC Score: 0.604 Average Precision: 0.676 Total-Time: 3
Step:  10/400 Loss: 7.93631 ROC-AUC Score: 0.710 Average Precision: 0.776 Total-Time: 4
Step:  12/400 Loss: 7.91557 ROC-AUC Score: 0.777 Average Precision: 0.830 Total-Time: 5
Step:  14/400 Loss: 7.87239 ROC-AUC Score: 0.821 Average Precision: 0.861 Total-Time: 5
Step:  16/400 Loss: 7.79032 ROC-AUC Score: 0.831 Average Precision: 0.867 Total-Time: 6
Step:  18/400 Loss: 7.65986 ROC-AUC Score: 0.838 Average Precision: 0.872 Total-Time: 7
Step:  20/400 Loss: 7.50709 ROC-AUC Score: 0.865 Average Precision: 0.890 Total-Time: 8
Step:  22/400 Loss: 7.30607 ROC-AUC Score: 0.877 Average Precision: 0.900 Total-Time: 8
Step:  24/400 Loss: 7.04079 ROC-

#### our CELL

In [8]:
method = 'nonlin cell'
optimizer_args = dict(lr=0.1, weight_decay=5e-5)
options = dict(g_type='fc', criterion='val', optimizer_args=optimizer_args, collect_stat=False)

run(**options)
#df[method] = stat_df

Step:   2/200 Loss: 7.96727 ROC-AUC Score: 0.804 Average Precision: 0.818 Total-Time: 1
Step:   4/200 Loss: 8.41841 ROC-AUC Score: 0.821 Average Precision: 0.812 Total-Time: 1
Step:   6/200 Loss: 7.58843 ROC-AUC Score: 0.823 Average Precision: 0.826 Total-Time: 2
Step:   8/200 Loss: 7.38567 ROC-AUC Score: 0.845 Average Precision: 0.846 Total-Time: 3
Step:  10/200 Loss: 7.27044 ROC-AUC Score: 0.849 Average Precision: 0.868 Total-Time: 4
Step:  12/200 Loss: 7.24348 ROC-AUC Score: 0.845 Average Precision: 0.870 Total-Time: 5
Step:  14/200 Loss: 7.15488 ROC-AUC Score: 0.853 Average Precision: 0.876 Total-Time: 6
Step:  16/200 Loss: 7.01889 ROC-AUC Score: 0.863 Average Precision: 0.883 Total-Time: 7
Step:  18/200 Loss: 6.83287 ROC-AUC Score: 0.865 Average Precision: 0.869 Total-Time: 8
Step:  20/200 Loss: 6.65570 ROC-AUC Score: 0.879 Average Precision: 0.887 Total-Time: 9
Step:  22/200 Loss: 6.45913 ROC-AUC Score: 0.887 Average Precision: 0.894 Total-Time: 10
Step:  24/200 Loss: 6.31568 ROC

# Citeseer

In [32]:
_A_obs, _X_obs, _z_obs = utils.load_npz('../data/citeseer.npz')
_A_obs = _A_obs + _A_obs.T
_A_obs[_A_obs > 1] = 1
_A_obs = _A_obs - sp.eye(_A_obs.shape[0], _A_obs.shape[0])
_A_obs[_A_obs < 0] = 0
lcc = utils.largest_connected_components(_A_obs)
_A_obs = _A_obs[lcc,:][:,lcc]
_N = _A_obs.shape[0]

Selecting 1 largest connected components


In [33]:
val_share = 0.05
test_share = 0.1
seed = 48

#there are self loops!

train_ones, val_ones, val_zeros, test_ones, test_zeros = utils.train_val_test_split_adjacency(_A_obs, val_share, test_share, seed, undirected=True, connected=True, asserts=False)

train_graph = sp.coo_matrix((np.ones(len(train_ones)),(train_ones[:,0], train_ones[:,1]))).tocsr()
assert (train_graph.toarray() == train_graph.toarray().T).all()

In [11]:
df = pd.DataFrame()

#### CELL

In [12]:
method = 'cell'
options = dict(g_type='cell')

stat_df, _ = run(**options)
df[method] = stat_df

Step:  10/200 Loss: 5.32038 Edge-Overlap: 0.040 Total-Time: 2
Step:  20/200 Loss: 3.01292 Edge-Overlap: 0.341 Total-Time: 5
Step:  30/200 Loss: 2.13118 Edge-Overlap: 0.596 Total-Time: 7
ROC-AUC: 0.8454040642722118, AVG_PREC: 0.8784820025495194


#### SVD

In [13]:
method = 'svd'
options = dict(g_type='svd', nsteps=400)

stat_df, _ = run(**options)
df[method] = stat_df

Step:  10/400 Loss: 7.64836 Edge-Overlap: 0.003 Total-Time: 3
Step:  20/400 Loss: 7.05292 Edge-Overlap: 0.030 Total-Time: 6
Step:  30/400 Loss: 5.37081 Edge-Overlap: 0.120 Total-Time: 9
Step:  40/400 Loss: 3.71165 Edge-Overlap: 0.274 Total-Time: 12
Step:  50/400 Loss: 2.85903 Edge-Overlap: 0.462 Total-Time: 15
Step:  60/400 Loss: 2.48062 Edge-Overlap: 0.579 Total-Time: 17
ROC-AUC: 0.9163368383742913, AVG_PREC: 0.9317245283055556


#### our CELL

In [14]:
method = 'nonlin cell'
optimizer_args = dict(lr=0.05, weight_decay=1e-5)
options = dict(g_type='fc', optimizer_args=optimizer_args)

stat_df, _ = run(**options)
df[method] = stat_df

Step:  10/200 Loss: 6.12156 Edge-Overlap: 0.030 Total-Time: 3
Step:  20/200 Loss: 3.49128 Edge-Overlap: 0.297 Total-Time: 6
Step:  30/200 Loss: 2.43985 Edge-Overlap: 0.569 Total-Time: 9
ROC-AUC: 0.873390241020794, AVG_PREC: 0.8912023379662789


#### Results

In [15]:
df['gt'] = list(compute_graph_statistics(_A_obs).values()) + [1, 1]

In [16]:
df

,cell,svd,nonlin cell,gt
d_max,48.600000,57.000000,52.400000,99.000000
d_min,1.000000,1.000000,1.000000,1.000000
d,2.954502,2.954502,2.954502,3.476777
LCC,2078.400000,2095.600000,2052.400000,2110.000000
wedge_count,13103.200000,13654.600000,15172.400000,25943.000000
claw_count,50562.800000,60357.200000,68962.600000,250348.000000
triangle_count,119.800000,165.800000,166.000000,1083.000000
square_count,350.000000,532.600000,510.200000,5977.000000
power_law_exp,2.186799,2.194704,2.253869,2.068238
gini,0.376411,0.380979,0.412664,0.428260


In [17]:
method = 'cell'
options = dict(g_type='cell', criterion='val', collect_stat=False)

run(**options)
#df[method] = stat_df

Step:   2/200 Loss: 7.63213 ROC-AUC Score: 0.611 Average Precision: 0.628 Total-Time: 0
Step:   4/200 Loss: 7.37223 ROC-AUC Score: 0.671 Average Precision: 0.682 Total-Time: 1
Step:   6/200 Loss: 6.84857 ROC-AUC Score: 0.702 Average Precision: 0.721 Total-Time: 1
Step:   8/200 Loss: 6.08958 ROC-AUC Score: 0.727 Average Precision: 0.747 Total-Time: 2
Step:  10/200 Loss: 5.25588 ROC-AUC Score: 0.755 Average Precision: 0.776 Total-Time: 2
Step:  12/200 Loss: 4.51554 ROC-AUC Score: 0.784 Average Precision: 0.807 Total-Time: 3
Step:  14/200 Loss: 3.96539 ROC-AUC Score: 0.809 Average Precision: 0.833 Total-Time: 3
Step:  16/200 Loss: 3.56039 ROC-AUC Score: 0.824 Average Precision: 0.848 Total-Time: 4
Step:  18/200 Loss: 3.21030 ROC-AUC Score: 0.829 Average Precision: 0.851 Total-Time: 5
Step:  20/200 Loss: 2.92053 ROC-AUC Score: 0.834 Average Precision: 0.854 Total-Time: 5
Step:  22/200 Loss: 2.68678 ROC-AUC Score: 0.844 Average Precision: 0.861 Total-Time: 5
Step:  24/200 Loss: 2.48397 ROC-

In [18]:
method = 'svd'
options = dict(g_type='svd', criterion='val', nsteps=400, collect_stat=False)

run(**options)
#df[method] = stat_df

Step:   2/400 Loss: 7.65649 ROC-AUC Score: 0.501 Average Precision: 0.522 Total-Time: 0
Step:   4/400 Loss: 7.65482 ROC-AUC Score: 0.543 Average Precision: 0.534 Total-Time: 0
Step:   6/400 Loss: 7.65470 ROC-AUC Score: 0.540 Average Precision: 0.580 Total-Time: 1
Step:   8/400 Loss: 7.65410 ROC-AUC Score: 0.613 Average Precision: 0.685 Total-Time: 1
Step:  10/400 Loss: 7.65079 ROC-AUC Score: 0.705 Average Precision: 0.769 Total-Time: 2
Step:  12/400 Loss: 7.63577 ROC-AUC Score: 0.748 Average Precision: 0.809 Total-Time: 2
Step:  14/400 Loss: 7.59685 ROC-AUC Score: 0.797 Average Precision: 0.847 Total-Time: 3
Step:  16/400 Loss: 7.51154 ROC-AUC Score: 0.838 Average Precision: 0.869 Total-Time: 3
Step:  18/400 Loss: 7.35529 ROC-AUC Score: 0.853 Average Precision: 0.880 Total-Time: 4
Step:  20/400 Loss: 7.15631 ROC-AUC Score: 0.877 Average Precision: 0.898 Total-Time: 4
Step:  22/400 Loss: 6.95681 ROC-AUC Score: 0.887 Average Precision: 0.909 Total-Time: 5
Step:  24/400 Loss: 6.69340 ROC-

In [34]:
method = 'nonlin cell'
optimizer_args = dict(lr=0.1, weight_decay=1e-5)
options = dict(g_type='fc', criterion='val', optimizer_args=optimizer_args, collect_stat=False)

run(**options)
#df[method] = stat_df

Step:   2/200 Loss: 7.62623 ROC-AUC Score: 0.772 Average Precision: 0.798 Total-Time: 0
Step:   4/200 Loss: 7.28031 ROC-AUC Score: 0.768 Average Precision: 0.803 Total-Time: 0
Step:   6/200 Loss: 7.09677 ROC-AUC Score: 0.769 Average Precision: 0.808 Total-Time: 1
Step:   8/200 Loss: 6.69518 ROC-AUC Score: 0.785 Average Precision: 0.820 Total-Time: 1
Step:  10/200 Loss: 6.23125 ROC-AUC Score: 0.765 Average Precision: 0.809 Total-Time: 2
Step:  12/200 Loss: 5.69926 ROC-AUC Score: 0.776 Average Precision: 0.811 Total-Time: 2
Step:  14/200 Loss: 5.18444 ROC-AUC Score: 0.781 Average Precision: 0.811 Total-Time: 3
Step:  16/200 Loss: 4.88091 ROC-AUC Score: 0.778 Average Precision: 0.791 Total-Time: 3
Step:  18/200 Loss: 4.40904 ROC-AUC Score: 0.814 Average Precision: 0.829 Total-Time: 4
Step:  20/200 Loss: 4.02828 ROC-AUC Score: 0.819 Average Precision: 0.839 Total-Time: 4
Step:  22/200 Loss: 3.69094 ROC-AUC Score: 0.821 Average Precision: 0.838 Total-Time: 5
Step:  24/200 Loss: 3.32875 ROC-

# Prolblogs

In [20]:
_A_obs, _X_obs, _z_obs = utils.load_npz('../data/polblogs.npz')
_A_obs = _A_obs + _A_obs.T
_A_obs[_A_obs > 1] = 1
_A_obs = _A_obs - sp.eye(_A_obs.shape[0], _A_obs.shape[0])
_A_obs[_A_obs < 0] = 0
lcc = utils.largest_connected_components(_A_obs)
_A_obs = _A_obs[lcc,:][:,lcc]
_N = _A_obs.shape[0]

Selecting 1 largest connected components


In [21]:
val_share = 0.05
test_share = 0.1
seed = 48

train_ones, val_ones, val_zeros, test_ones, test_zeros = utils.train_val_test_split_adjacency(_A_obs, val_share, test_share, seed, undirected=True, connected=True, asserts=False)

train_graph = sp.coo_matrix((np.ones(len(train_ones)),(train_ones[:,0], train_ones[:,1]))).tocsr()
assert (train_graph.toarray() == train_graph.toarray().T).all()

In [22]:
df = pd.DataFrame()

#### CELL

In [23]:
method = 'cell'
options = dict(g_type='cell')

stat_df, _ = run(**options)
df[method] = stat_df

Step:  10/200 Loss: 5.85307 Edge-Overlap: 0.275 Total-Time: 1
Step:  20/200 Loss: 5.48650 Edge-Overlap: 0.294 Total-Time: 3
Step:  30/200 Loss: 5.32603 Edge-Overlap: 0.324 Total-Time: 4
Step:  40/200 Loss: 5.24834 Edge-Overlap: 0.338 Total-Time: 6
Step:  50/200 Loss: 5.20862 Edge-Overlap: 0.346 Total-Time: 8
Step:  60/200 Loss: 5.18560 Edge-Overlap: 0.343 Total-Time: 9
Step:  70/200 Loss: 5.17081 Edge-Overlap: 0.353 Total-Time: 11
Step:  80/200 Loss: 5.15981 Edge-Overlap: 0.350 Total-Time: 12
Step:  90/200 Loss: 5.15128 Edge-Overlap: 0.357 Total-Time: 14
Step: 100/200 Loss: 5.14501 Edge-Overlap: 0.359 Total-Time: 16
Step: 110/200 Loss: 5.14072 Edge-Overlap: 0.361 Total-Time: 17
Step: 120/200 Loss: 5.13775 Edge-Overlap: 0.364 Total-Time: 19
Step: 130/200 Loss: 5.13552 Edge-Overlap: 0.359 Total-Time: 21
Step: 140/200 Loss: 5.13377 Edge-Overlap: 0.361 Total-Time: 22
Step: 150/200 Loss: 5.13235 Edge-Overlap: 0.356 Total-Time: 24
Step: 160/200 Loss: 5.13123 Edge-Overlap: 0.357 Total-Time: 2

#### SVD

In [24]:
method = 'svd'
options = dict(g_type='svd', nsteps=400)

stat_df, _ = run(**options)
df[method] = stat_df

Step:  10/400 Loss: 7.09045 Edge-Overlap: 0.020 Total-Time: 1
Step:  20/400 Loss: 6.43193 Edge-Overlap: 0.150 Total-Time: 2
Step:  30/400 Loss: 5.85576 Edge-Overlap: 0.241 Total-Time: 4
Step:  40/400 Loss: 5.62244 Edge-Overlap: 0.275 Total-Time: 5
Step:  50/400 Loss: 5.47940 Edge-Overlap: 0.300 Total-Time: 7
Step:  60/400 Loss: 5.40886 Edge-Overlap: 0.327 Total-Time: 8
Step:  70/400 Loss: 5.37150 Edge-Overlap: 0.314 Total-Time: 10
Step:  80/400 Loss: 5.36684 Edge-Overlap: 0.325 Total-Time: 11
Step:  90/400 Loss: 5.34928 Edge-Overlap: 0.330 Total-Time: 13
Step: 100/400 Loss: 5.32909 Edge-Overlap: 0.322 Total-Time: 14
Step: 110/400 Loss: 5.32822 Edge-Overlap: 0.318 Total-Time: 16
Step: 120/400 Loss: 5.32070 Edge-Overlap: 0.337 Total-Time: 18
Step: 130/400 Loss: 5.31531 Edge-Overlap: 0.328 Total-Time: 19
Step: 140/400 Loss: 5.31278 Edge-Overlap: 0.340 Total-Time: 21
Step: 150/400 Loss: 5.30887 Edge-Overlap: 0.331 Total-Time: 23
Step: 160/400 Loss: 5.29684 Edge-Overlap: 0.340 Total-Time: 2

#### our CELL

In [28]:
method = 'nonlin cell'
optimizer_args={'lr': 0.1, 'weight_decay': 1e-5}
options = dict(g_type='fc', optimizer_args=optimizer_args)

stat_df, _ = run(**options)
df[method] = stat_df

Step:  10/200 Loss: 6.48913 Edge-Overlap: 0.083 Total-Time: 1
Step:  20/200 Loss: 5.95763 Edge-Overlap: 0.203 Total-Time: 3
Step:  30/200 Loss: 5.72736 Edge-Overlap: 0.232 Total-Time: 5
Step:  40/200 Loss: 5.61247 Edge-Overlap: 0.254 Total-Time: 6
Step:  50/200 Loss: 5.51633 Edge-Overlap: 0.275 Total-Time: 8
Step:  60/200 Loss: 5.42756 Edge-Overlap: 0.299 Total-Time: 10
Step:  70/200 Loss: 5.37747 Edge-Overlap: 0.305 Total-Time: 11
Step:  80/200 Loss: 5.30751 Edge-Overlap: 0.325 Total-Time: 13
Step:  90/200 Loss: 5.26652 Edge-Overlap: 0.337 Total-Time: 15
Step: 100/200 Loss: 5.22512 Edge-Overlap: 0.337 Total-Time: 16
Step: 110/200 Loss: 5.20817 Edge-Overlap: 0.346 Total-Time: 18
Step: 120/200 Loss: 5.16152 Edge-Overlap: 0.358 Total-Time: 19
Step: 130/200 Loss: 5.17501 Edge-Overlap: 0.352 Total-Time: 21
Step: 140/200 Loss: 5.14056 Edge-Overlap: 0.369 Total-Time: 23
Step: 150/200 Loss: 5.12188 Edge-Overlap: 0.364 Total-Time: 24
Step: 160/200 Loss: 5.14070 Edge-Overlap: 0.367 Total-Time: 

In [29]:
df['gt'] = list(compute_graph_statistics(_A_obs).values()) + [1, 1]

In [30]:
df

,cell,svd,gt,nonlin cell
d_max,2.602000e+02,2.640000e+02,3.510000e+02,3.044000e+02
d_min,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
d,2.325041e+01,2.325041e+01,2.735516e+01,2.325041e+01
LCC,1.222000e+03,1.221000e+03,1.222000e+03,1.221600e+03
wedge_count,8.504556e+05,8.771874e+05,1.341525e+06,9.356972e+05
claw_count,2.892156e+07,3.073516e+07,6.280078e+07,3.580949e+07
triangle_count,4.073520e+04,4.348720e+04,1.010430e+05,4.813580e+04
square_count,1.565213e+06,1.715766e+06,5.171257e+06,1.989580e+06
power_law_exp,1.407178e+00,1.413679e+00,1.414274e+00,1.427987e+00
gini,5.778448e-01,5.890060e-01,6.220195e-01,6.083670e-01
